In [1]:

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

** Load Data**

In [2]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


**Image Preprocessing**

In [3]:

## check existing image size
train_ds[0].shape
     

TensorShape([442, 1024, 3])

In [4]:

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [5]:

train_labels
     

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0])>

In [6]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [7]:

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [8]:
train_labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

**Use Pretrained VGG16 Image Classification model**

In [9]:

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [10]:

train_ds[0].shape

TensorShape([150, 150, 3])

In [11]:

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
## will not train base mode
base_model.trainable = False 

In [13]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)
     

In [14]:
## model details
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [15]:
#add our layers on top of this model
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])
     

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [17]:

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

In [ ]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 687s 11s/step - loss: 1.9517 - accuracy: 0.4701 - val_loss: 1.2710 - val_accuracy: 0.5233
Epoch 2/50
65/65 [==============================] - 681s 11s/step - loss: 0.8428 - accuracy: 0.6822 - val_loss: 1.0036 - val_accuracy: 0.6401
Epoch 3/50
65/65 [==============================] - 682s 11s/step - loss: 0.5602 - accuracy: 0.7849 - val_loss: 0.9709 - val_accuracy: 0.6770
Epoch 4/50
65/65 [==============================] - 678s 10s/step - loss: 0.3856 - accuracy: 0.8530 - val_loss: 1.0091 - val_accuracy: 0.6868
Epoch 5/50
65/65 [==============================] - 680s 11s/step - loss: 0.3107 - accuracy: 0.8871 - val_loss: 1.0707 - val_accuracy: 0.6848
Epoch 6/50
65/65 [==============================] - 678s 10s/step - loss: 0.2239 - accuracy: 0.9148 - val_loss: 1.0716 - val_accuracy: 0.7062
Epoch 7/50
65/65 [==============================] - 681s 11s/step - loss: 0.1861 - accuracy: 0.9392 - val_loss: 1.1408 - val_accuracy: 0.6946
Epoch 